In [1]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optm
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

from sklearn.metrics import f1_score
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib import gridspec

from torchinfo import summary
from tqdm.auto import tqdm

%matplotlib inline
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cwd=os.path.dirname(os.getcwd())

In [16]:
def make_images(meta,img_dir,train):
    images=[]
    labels=[]
    if train:
        for idx in range(len(meta)):
            folder_path=os.path.join(img_dir, meta.path[idx])
            for img in os.listdir(folder_path):
                if '._' in img or '.ipynb' in img:
                    continue
                images.append(os.path.join(folder_path,img))
                labels.append((('incorrect' in img)+('normal' in img)*2)*6+(meta.gender[idx]=='female')*3+(30<=meta.age[idx])+(60<=meta.age[idx]))
    else:
        for img_id in meta.ImageID:
            images.append(os.path.join(img_dir, img_id))
    return images,labels

trans=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class ImageDataset(Dataset):
    def __init__(self,transform=trans,train=True):
        self.train=train
        self.md=['info','train']
        self.path=[os.path.join(cwd,'input/data/eval'),os.path.join(cwd,'input/data/train')]
        self.meta=pd.read_csv(os.path.join(self.path[train], f'{self.md[train]}.csv'))
        self.img_dir=os.path.join(self.path[train],'images')
        self.classes=[('Wear','Incorrect','Not Wear'),('남','여'),('<30','>=30 and <60','>=60')]
        self.trans=trans
        
        self.images,self.labels=make_images(self.meta,self.img_dir,train)

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image=Image.open(self.images[idx])
        image=self.trans(image)
        if train:
            label=self.labels[idx]
        else:
            label=0
        return image,label


In [3]:
#사용할 부분 모델 불러오기
net_mask = torchvision.models.resnext50_32x4d(pretrained=True)
net_gender = torchvision.models.resnext50_32x4d(pretrained=True)
net_age = torchvision.models.resnet50(pretrained=True)

#초기화 하기
net_mask.fc=torch.nn.Linear(in_features=2048, out_features=3, bias=True)
net_gender.fc=torch.nn.Linear(in_features=2048, out_features=2, bias=True)
net_age.fc=torch.nn.Linear(in_features=2048, out_features=3, bias=True)


torch.nn.init.xavier_uniform_(net_mask.fc.weight)
torch.nn.init.xavier_uniform_(net_gender.fc.weight)
torch.nn.init.xavier_uniform_(net_age.fc.weight)

stdv=1/np.sqrt(2048)
net_mask.fc.bias.data.uniform_(-stdv, stdv)
net_gender.fc.bias.data.uniform_(-stdv, stdv)
net_age.fc.bias.data.uniform_(-stdv, stdv)

tensor([-0.0008, -0.0153, -0.0126])

In [4]:
#모델 구축
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.mask=net_mask
        self.gender=net_gender
        self.age=net_age
        
    def forward(self,x):
        mask=self.mask(x).view(x.size(0),3,1,1)
        gender=self.gender(x).view(x.size(0),1,2,1)
        age=self.age(x).view(x.size(0),1,1,3)
        return (mask*gender*age).view(x.size(0),-1)

In [ ]:
#모델 설정하기
model_dir=os.path.join(os.getcwd(),'saved/total_model/')
if os.listdir(model_dir):
    latest_model_id=sorted(os.listdir(model_dir))[-1]
    model=torch.load(os.path.join(model_dir,latest_model_id))
    add_epoch=int(latest_model_id.split('_')[2])+1
else:
    model=MyModel()


In [5]:
#학습 설정하기
batch_size=100
learning_rate=0.001

optim=optm.Adam(model.parameters())
loss_fn=nn.CrossEntropyLoss()

In [ ]:
train=False
model.to(device)
    
data=ImageDataset(train=train)

if train:
    dataloader=DataLoader(data,batch_size=batch_size,shuffle=True,num_workers=4)
    model.train()
else:
    submission = pd.read_csv(os.path.join(data.path[0], 'info.csv'))
    dataloader=DataLoader(data,shuffle=False)
    model.eval()
    
with tqdm(dataloader) as pbar:
    running_loss=0.
    running_acc=0.
    tot_pred=torch.tensor([]).to(device)
    tot_label=torch.tensor([]).to(device)
    for n,(image,label) in enumerate(pbar):
        image=image.to(device)
        label=label.to(device)
            
        logit=model(image)
        _,pred=torch.max(logit,1)
            
        if train:
            optim.zero_grad()
            loss=loss_fn(logit,label)
            loss.backward()
            optim.step()
            running_loss+=loss.item()*image.size(0)
            running_acc+=torch.sum(pred==label)
            pbar.set_postfix({'epoch' : epoch+1, 'loss' : running_loss/(n+1), 'accuracy' : float(running_acc)/(n+1),'F1 score':f1_score(label.cpu(),pred.cpu(),average='weighted')})
            
        tot_pred=torch.hstack((tot_pred,pred))
        tot_label=torch.hstack((tot_label,label))
    if train:
        epoch_loss=running_loss/len(dataloader.dataset)
        epoch_acc=running_acc/len(dataloader.dataset)
        print(f"현재 epoch-{epoch+1}의 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}, F1 score : {f1_score(tot_label.cpu(),tot_pred.cpu(),average='weighted')}" )
        torch.save(model,f"{model_dir}total_model_{add_epoch}_{epoch_loss:.3f}_{epoch_acc:.3f}_{f1_score(tot_label.cpu(),tot_pred.cpu(),average='weighted')}.pt")
    else:
        submission['ans'] = tot_pred.cpu().numpy().astype('int64')
        submission.to_csv(os.path.join(data.path[0], f'submission_{add_epoch}.csv'), index=False)